##Term Project - Understanding the Impact of various intervention strategies on Building’s                                                    Energy Usage through Agent Based Models
### Author: Albert Thomas and Bharadwaj Mantha 
#### Complex Systems 530 - Computer Modeling of Complex Systems (Winter 2015)
  * Course ID: CMPLXSYS 530
  * Course Title: Computer Modeling of Complex Systems
  * Term: Winter 2015 
  * Date modified - 3_20_2015
  * Time 10:50 PM

# 1. Problem Statement

The main objective of this project is to create an agent based model, that simulates the effects of dynamic occupant interactions and interventions such as variations in the building performance levels and frequent education programs in reducing the energy usage of the building. 

## 2. Why ABM for solving the problem

There are dynamic occupant interactions and building manager interventions during the life time of a building which can affect the energy usage patterns in a building. To represent this dynamic nature, ABM provides the best solution. 

In this notebook, we will develop a simple model that:
* uses object-oriented concepts, e.g., classes
* implements an agent-based approach to Building Energy Modeling
* demonstrates how an institution or norm set by Building Manager can influence the building systems efficiency and the energy use patterns of occupants. 

## 3. Defining the Model

### Space

In this model, our space will be a two-dimensional (2D) square grid. Each grid cell will contain zero or one people. Edges of the grid will wrap around.

### Actors 

#### Agents

This Agent Based Model will have two different types of agents, 
* Building Occupants - who live and interact in the space (building) and
* Building Managers - who are responsible for the dynamic interventions such as educational programs, building maintenance etc.

#### Institution

In this model Building manager (one of the agents in this model) sets the institutions/ norms i.e. the frequency of maintenance activities and educational awareness programs. 

### Initial conditions

#### Agents

* Building occupants will be randomly distributed throughout the grid by sampling from a uniform discrete distribution with replacement. In the event that an occupant has already been placed at the sampled location, we continue drawing a new nandom position until it is unoccupied.
* Building occupants will have their probability of interaction randomly initialized to a value from any continuous distribution.
* Building Managers have their probability of interventions randomly initialized. 

#### Institutions

* The building energy efficiency set to a default level of 100% at the start of the simualation.

### Model Parameters

Based on the description above, we need the following model parameters:

 * ``grid_size``: size of the two-dimensional square grid, i.e., dimension length of the space
 * ``population_size``: number of occupants in the building; must be less than ${grid\_size}^2$
 * `` prob_interaction``: prabability of occupants interacting. 
 
### List of sweep parameters
 
 * ``population_size`` : keeping the space constant how population_size changes the energy usage in buildings
 * ``user_type`` : Initial percentage of low, medium, and high energy users
 * `` maintenance_frequency`` : The frequency of maintenance of the building
 * ``affect_AP `` affect of awareness programs on the occupants

In [2]:
# Scientific computing imports
import numpy 
import numpy.random

class Building_Occupants():
    
    """
    Occipant class, which encapsulates the entire behavior of an occupant in a building.
    
    """
    
        
    def __init__ (self, model, Occupant_id, user_type = "Medium_USer"
                  ):
        """
        Constructor for Building_Occupants class
        By default,
        user_type = "Medium_User" i.e. medium energy user. 
    
        """
        self.model = model
        self.user_type = user_type
        self.Occupant_id = Occupant_id

    def get_User_Type(self):
        
        # This method gets the Occupant type  
        
        random_number = numpy.random.randint(0,100)
        if (0 < random_number <= 33):
            user_type = "Low_User"
            return user_type
        elif (33 < random_number <= 66):
            user_type = "Medium_User"
            return user_type
        else:
            user_type = "High_User"
            return user_type
        
    def Occupant_energy_intensity(self):
        
        # This method gets the energy intensity based on the occupant type 
        
        if (self.user_type == "Low_User"):
            return 200
        elif (self.user_type == "Medium_USer"):
            return 300
        else:
            return 400
        

#Agent_type = Building_Agents()
#print Agent_type.get_User_Type()
        

In [56]:
# Scientific computing imports
import numpy 

class Model(object):
    """
    Model class, which encapsulates the entire behavior of a single "run" in our HIV ABM.
    
    """
    def __init__(self,grid_size, population_size, numOfYears):
        
        # Setting the model parameters;
        self.grid_size = grid_size
        self.population_size = population_size
        self.numOfYears = numOfYears
        
        # Setting the state variables
        self.Occupants = []
    
    ######################################## Changes MADE on 03-30-15#########################
    def bldgEfficiency_decrease(self): 
        """
        This method generates the yearly decrease in building efficiency.
        A building's efficiency is determined by the systems efficiency and the material property variations 
        """
        syst_eff_dec = numpy.random.uniform(0.5,1.5) # Building systems normally have the biggest influence on the eff
        mat_eff_dec = numpy.random.uniform(0,0.5)
        
        bldg_eff_dec = syst_eff_dec + mat_eff_dec
        
        return bldg_eff_dec
    
    
    def bldgWalls_Maintenance(self):
        """
        This method generates increase in building efficiency due to maintenance of building walls
        """
        return numpy.random.uniform(0.1,1.5)
    
    def bldgRoofs_Maintenance(self):
        
        """
        This method generates increase in building efficiency due to maintenance of building roofs
        """
        return numpy.random.uniform(0.1,0.5)
   
        
    ######################################## Changes MADE on 03-30-15#########################
    
    def edu_Programs(self):
        """
        This function decides the frequency of educational awareness programs stochastically 
        """
        # creating an empty list 
        edu_List= []
        
        for i in range(self.numOfYears):
            edu_List.append(0)
        for i in range(self.numOfYears):
            edu_List[i]= numpy.random.binomial(1,0.5)
       
        return edu_List 
    # If the output cell value is 1, that means there is an educational progarm 
    # for that particular year
        
    def setup_space(self):
        """
        Method to setup the space.
        """
        # Initialize a space with a NaN's
        self.space = numpy.full((self.grid_size, self.grid_size), numpy.nan)
        print self.space    

    def setup_Occupants(self):
        """
        Method to setup the Occupants in the space.
        """
        # First, begin by creating all agents without placing them.
        for i in xrange(self.population_size):
            self.Occupants.append(Building_Occupants(model=self, user_type = "Medium_User" ,Occupant_id = i))
            
        # Second, once created, place them into the space.
        for Occupants in self.Occupants:
            # Loop until unique
            is_occupied = True
            while is_occupied:
                # Sample location
                random_x = numpy.random.randint(0, self.grid_size)
                random_y = numpy.random.randint(0, self.grid_size)
                
                # Check if unique
                if numpy.isnan(self.space[random_x, random_y]):
                    is_occupied = False
                else:
                    is_occupied = True
            
            # Now place the occupants there by setting their ID.
            self.space[random_x, random_y] = Occupants.Occupant_id
        print self.space
        
    def get_occupant_position(self, Occupant_id):
        """
        Get the position of an occupant based on their ID.
        """
        # Find the value that matches the occupant_id in self.space, then reshape to a 2-element list.
        return numpy.reshape(numpy.where(m.space == Occupant_id), (1, 2))[0].tolist()
    
    def move_occupant(self, occupant_id, x, y):
        """
        Move a person to a new (x, y) location.
        """
        
        # Get original
        original_position = self.get_occupant_position(Occupant_id)
        
        # Check target location
        if not numpy.isnan(self.space[x, y]):
            raise ValueError("Unable to move occupant {0} to ({1}, {2}) since occupied.".format(Occupant_id, x, y))
        
        # Otherwise, move by emptying and setting.
        self.space[original_position[0], original_position[1]] = numpy.nan
        self.space[x, y] = person_id
    
    
    
    def step_move(self):
        """
        Model step move function, which handles moving occupants randomly around.
        """
        
        # Get a random order for the agents.
        random_order = range(self.population_size)
        numpy.random.shuffle(random_order) # inplace method
        
        # Iterate in random order.
        for i in random_order:
            # Get current position
            x, y = self.get_occupant_position(i)
            
            # Move the occupant between -1, 0, +1 in each dimension
            x_new = (x + numpy.random.randint(-1, 2)) % self.grid_size
            y_new = (y + numpy.random.randint(-1, 2)) % self.grid_size
            
            # Try to move them
            try:
                self.move_occupant(i, x_new, y_new)
            except ValueError:
                # Occupied, so fail.
                pass
  
 ######################################## Changes MADE on 03-30-15#########################
    
    def step(self, wallMfreq, roofMfreq):
        """
        Model step function. 
        """
    
        bldgEfficiencyList =[]
        bldgEfficiencyList.append(100)
        
        for i in range(1,self.numOfYears):
            print i 
            if i % wallMfreq ==0 and i % roofMfreq == 0 :
                bldgEfficiencyList.append (bldgEfficiencyList[i-1] - self.bldgEfficiency_decrease()+ self.bldgWalls_Maintenance()+ self.bldgRoofs_Maintenance())
                print 'both' ## These print statements just for understanding and later remove
            elif i % wallMfreq == 0:    
                bldgEfficiencyList.append (bldgEfficiencyList[i-1] - self.bldgEfficiency_decrease()+ self.bldgWalls_Maintenance())
                print 'only wall maintenance'
            elif i % roofMfreq == 0:    
                bldgEfficiencyList.append (bldgEfficiencyList[i-1] - self.bldgEfficiency_decrease()+ self.bldgRoofs_Maintenance())
                print 'only roof maintenance'
            else:
                bldgEfficiencyList.append (bldgEfficiencyList[i-1] - self.bldgEfficiency_decrease())        
                print 'no maintenance'
        print "Buiilding performance Variation Pattern: \n"
        print bldgEfficiencyList     

######################################## Changes MADE on 03-30-15#########################

m=Model(grid_size = 3, population_size = 3, numOfYears = 10)
m.setup_space()
m.setup_Occupants()
m.step(2,5) # Here I have just given numbers now. When parameter comes it can be passed into the step function.
print "\n Presence of Educational Awareness Programs \n"
print m.edu_Programs()

[[ nan  nan  nan]
 [ nan  nan  nan]
 [ nan  nan  nan]]
[[  1.  nan  nan]
 [  0.   2.  nan]
 [ nan  nan  nan]]
1
no maintenance
2
only wall maintenance
3
no maintenance
4
only wall maintenance
5
only roof maintenance
6
only wall maintenance
7
no maintenance
8
only wall maintenance
9
no maintenance
Buiilding performance Variation Pattern: 

[100, 99.40418541449381, 99.71892184245486, 98.10964827406983, 97.3382012803544, 96.01120593417807, 94.95980335746444, 93.50131471035365, 94.21379291732576, 93.47482292767884]

 Presence of Educational Awareness Programs 

[1, 0, 0, 1, 0, 1, 0, 1, 1, 0]


## Future Works

Firstly, we plan to develop more methods which include interacting occupants and step method to finally compute the energy consumption alongside improving the existing framework we developed till date. In addition to that we will keep on developing the current framework. 

## Results

### Overview of the results

Created Model Space and allocated occupants in the space
Simulated the building performance variation pattern and the presence of educational programs over the lifetime of a building.
 


### Hypothesis of the results

One of the very obvious results we expect is by increasing the number of occupants in the space, building energy consumption increases. But we are interesting in looking at the results based on the initial percentage of the occupants (low, medium and high).  
